# DLH_utils demo

This notebook is intended to be a demo of what you *could* use DLH_utils for. 

I'm sure to many a lot of this code may look very similar! We will have taken similar approaches for the vast majority of the problems faced here. We've just wrapped these mostly standard approaches up into reusable functions, hopefully to save everyone doing linkage both some time and headaches! 

In [ ]:
# to start, install dlh_utils if not installed already. Notice the '-U' argument to upgrade existing installations. 
!pip3 install -U '/home/cdsw/dlh_utils'

In [ ]:
# import necessary libraries
import pyspark.sql.functions as F
import pandas as pd

from dlh_utils import utilities
from dlh_utils import dataframes
from dlh_utils import linkage
from dlh_utils import standardisation
from dlh_utils import sessions
from dlh_utils import profiling
from dlh_utils import flags

In [ ]:
# you can use our sessions module to set up your spark session
# this will also create a Spark UI, which you can use to track your code's efficiency
spark = sessions.getOrCreateSparkSession(appName = 'dlh_utils_demo', size = 'medium')

In [ ]:
# read in raw data
df1 = pd.read_csv()
df2 = pd.read_csv

# note, if this was stored in Hue, the read_format() function from the utilities module would've been useful

#for demo purposes, let's convert this to a spark df using utilities
df1 = utilities.pandas_to_spark(df1)
df2 = utilities.pandas_to_spark(df2)

To give a quick overview of the features of our data, we can use the **descibe()** function from the profiling module:

In [ ]:
descriptive_df1 = profiling.dab_describe(df1,
                                         mode = 'pandas',
                                         approx_distinct = False,
                                         rsd = 0.05
                                        )
descriptive_df1

From this we can see that we have a percentage distinct in our sex variable far from 50% which we would expect. This could suggest a high level of missingness, but we can see from the rest of the output that we don't have any missing or null sex values, suggesting some have been incorrectly coded or skewed in the data.

We also don't have many distinct postcodes, and our surname variable has a lot of 'empty' values. On bigger data, these observations can give quick insights into which variables may be the most/least useful for matching. 

The **value_counts()** functions shows the top or bottom n values in our data. This can give us an overview of the different types of missingness in these variables, which will be useful when we come to standardise missingness in our data later.

In [ ]:
top_5_value_counts = profiling.value_counts(df1,
                                            limit = 5,
                                            mode = 'pandas'
                                            )
# the value counts function returns two dataframes; one for the top n values in each variable and one for the bottom n values. 
# we can select the top value count dataframe by subsetting the top_5_value_counts_df1 tuple:

top_5_value_counts[0]

To flag out of scope values in our data, we can use the **flag()** function:

In [ ]:
out_of_scope = flags.flag(df = df1,
                          ref_col = 'DOB',
                          condition = '<=',
                          condition_value = '1900-00-00',
                          condition_col = None,
                          alias = None,
                          prefix = 'FLAG',
                          fill_null = None
                         )
out_of_scope.show()

We can see we have supercentenarian Ben in our data, which is probably wrong, but we've also got a few different date types that have been flagged as well. 

If you are working with larger data, the **flag_check()** and **flag_summary()** functions can produce more detailed flag metrics that will help you spot issues like this more readily. 

Let's move on to cleaning and standardising where we can start to deal with these issues.

In [ ]:
df1.show()

# Data Cleaning & Standardisation

In [ ]:
# replace '\n' values with spaces:

df1 = standardisation.reg_replace(df = df1, dic = {' ': '\n'})
df2 = standardisation.reg_replace(df = df2, dic = {' ': '\n'})

Let's standardise the date format to be consistent across our data in a **ddMMyyyy** format:

In [ ]:
df1 = standardisation.standardise_date(df1, col_name = "DOB", in_date_format = "yyyy-MM-dd", out_date_format = "dd/MM/yyyy")
df2 = standardisation.standardise_date(df2, col_name = "DOB", in_date_format = "yyyy-MM-dd", out_date_format = "dd/MM/yyyy")

df1.show()

Next, we have generic 'ID' columns in each dataset. We also have address and name variables named differently in each dataset. 

We can use **rename_columns()** from the dataframes module to rename all of these at once. 

In [ ]:
df1 = dataframes.rename_columns(df1, rename_dict = {"ID":"ID_DF1","ENUM_ADDR":"ADDRESS","ENUM_FNAME":"FORENAME","ENUM_SNAME":"SURNAME"})
df2 = dataframes.rename_columns(df2, rename_dict = {"ID":"ID_DF2","ENUM_ADDR":"ADDRESS","ENUM_FNAME":"FORENAME","ENUM_SNAME":"SURNAME"})

df1.columns

Now let's set all of variables to upper case for consistency, using **standardise_case()**:

In [ ]:
df1 = standardisation.standardise_case(df1)
df2 = standardisation.standardise_case(df2)

df1.show()

Next, the values for missingness are all over the place. I can spot a few NaNs, nulls, and whitespaces. Let's standardise missingness with the **standardise_null()** function. We can retrieve these null values from the previous **value_counts()** outputs: 

In [ ]:
# we can use the standardise_null function to replace these with true None values:
# we use regex to do this: https://regex101.com/ 
df1 = standardisation.standardise_null(df1, replace = "^NAN$|^NULL$|^\s*$|^----$|^####$")
df2 = standardisation.standardise_null(df2, replace = "^NAN$|^NULL$|^\s*$|^----$|^####$")

df1.show()

Great, these now all show up as true nulls. 

Next, we have a mix of 1s, 2s, Ms, and Fs in our sex column. Let's standardise this to be either 1s or 2s. For this we can use **reg_replace()**:

In [ ]:
# reg_replace() takes a dictionary, where the value is the regex to replace, and the key is what this will be replaced with
# so we're replacing 'M' with '1', and 'F' with '2':
df1 = standardisation.reg_replace(df1, subset = "SEX", dic = {"1":"^M$","2":"^F$"})
df2 = standardisation.reg_replace(df2, subset = "SEX", dic = {"1":"^M$","2":"^F$"})

df1.show()

Now that our sex column is populated with just 1s and 2s, we might want to change the type from string to integer. This can be done using the **cast_type()** function in the standardisation module:

In [ ]:
df1 = standardisation.cast_type(df1, subset = ['SEX'], types = "integer")
df2 = standardisation.cast_type(df2, subset = ['SEX'], types = "integer")

df1.select('SEX').dtypes

Let's begin to have a look at our postcode, address, and name variables now. It looks like we sometimes have whitespace in the postcode column, and sometimes have multiple consecutive whitespaces in our name/address columns. 

We can use the **standardise_white_space()** function to limit whitespace in all columns to one, then use it again on just the postcode column, by specifying the subset argument and limiting it to only the postcode column. 

In [ ]:
df1 = standardisation.standardise_white_space(df1, wsl = "one")
df2 = standardisation.standardise_white_space(df2, wsl = "one")

df1 = standardisation.standardise_white_space(df1, subset = "POSTCODE", wsl = "none")
df2 = standardisation.standardise_white_space(df2, subset = "POSTCODE", wsl = "none")

df1.show()

We still have some leading/trailing whitespace in some of our variables, let's **trim** these:

In [ ]:
df1 = standardisation.trim(df1)
df2 = standardisation.trim(df2)

df1.show()

Next, let's focus on our name variables. Forenames still contain titles and some surnames have common prefixes like 'Van' or 'Der'. We can strip out titles and concatenate surname prefixes with our **clean_forename()** and **clean_surname()** functions. 

In [ ]:
df1 = standardisation.clean_forename(df1, subset = 'FORENAME')
df2 = standardisation.clean_forename(df2, subset = 'FORENAME')

df1 = standardisation.clean_surname(df1, subset = 'SURNAME')
df2 = standardisation.clean_surname(df2, subset = 'SURNAME')

df1.show()

Finally, let's strip out numbers from our name variables. Again, we can use the **reg_replace()** function for this:

In [ ]:
df1 = standardisation.reg_replace(df1, subset = ["FORENAME","SURNAME"], dic = {"": "[0-9]"})
df2 = standardisation.reg_replace(df2, subset = ["FORENAME","SURNAME"], dic = {"": "[0-9]"})

df1.show()

This still leaves apostrophes and hyphens in our name variables. The **remove_punct()** function can handle these. While we're at it, let's also use **remove_punct()** to get rid of dashes in our address field, but we'll have to specify the optional argument **keep** to make sure it doesn't strip out commas from addresses:ADD IN SEPARATE FUNCTION CALL SO COMMAS REMOVED FROM NAMES - CAN USE A LOOP

In [ ]:
df1 = standardisation.remove_punct(df1, subset = ["FORENAME","SURNAME","ADDRESS"], keep = ",")
df2 = standardisation.remove_punct(df2, subset = ["FORENAME","SURNAME"], keep = ",")

df1.show()

# Derive Variables

We've got quite a few identifying variables that we can split out into further variables for matching. 

First, let's derive street and town from the address variable. The **split()** function from the dataframes module will be useful here, splitting on comma. 

In [ ]:
# this will create a new column called "ADDRESS_SPLIT" that contains an array of each address element, separated by a comma
df1 = dataframes.split(df1, col_in = "ADDRESS", col_out = "ADDRESS_SPLIT", split_on = ",")
df2 = dataframes.split(df2, col_in = "ADDRESS", col_out = "ADDRESS_SPLIT", split_on = ",")

df1.select("ADDRESS", "ADDRESS_SPLIT").show()

In [ ]:
# we can then select the first element of the 'split address' to create the 'street address' variable COULD WE USE LOOP?
df1 = dataframes.index_select(df1, split_col = "ADDRESS_SPLIT", out_col = "ADDRESS_STREET", index = 0)
df2 = dataframes.index_select(df2, split_col = "ADDRESS_SPLIT", out_col = "ADDRESS_STREET", index = 0)

# the second element contains the town name, which we can append to a new column also 
df1 = dataframes.index_select(df1, split_col = "ADDRESS_SPLIT", out_col = "ADDRESS_TOWN", index = 1)
df2 = dataframes.index_select(df2, split_col = "ADDRESS_SPLIT", out_col = "ADDRESS_TOWN", index = 1)

# since we no longer need the 'ADDRESS_SPLIT' column, we can remove it using our drop_columns() function
df1 = dataframes.drop_columns(df1, subset = 'ADDRESS_SPLIT')
df2 = dataframes.drop_columns(df2, subset = 'ADDRESS_SPLIT')

df1.select("ADDRESS", "ADDRESS_STREET", "ADDRESS_TOWN").show()

We can create a 'full name' variable by concatenating the two existing name columns together, using **concat()**:

In [ ]:
df1 = dataframes.concat(df1, cols = ["FORENAME", "SURNAME"], sep = " ", out_col = "FULL_NAME")
df2 = dataframes.concat(df2, cols = ["FORENAME", "SURNAME"], sep = " ", out_col = "FULL_NAME")

df1.select("FORENAME", "SURNAME", "FULL_NAME").show()

For ethnically diverse datasets, phonetic encodings of name variables may aid matching. We have functions for this in the linkage module. 

In [ ]:
df1 = df1.withColumn('FORENAME_METAPHONE', linkage.metaphone(F.col('FORENAME')))
df1 = df1.withColumn('FORENAME_SOUNDEX', linkage.soundex(F.col('FORENAME')))

df2 = df2.withColumn('FORENAME_METAPHONE', linkage.metaphone(F.col('FORENAME')))
df2 = df2.withColumn('FORENAME_SOUNDEX', linkage.soundex(F.col('FORENAME')))

df1.select("FORENAME", "FORENAME_METAPHONE", "FORENAME_SOUNDEX").show()  

Similarly, if there have been spelling mistakes, alphabetising string columns may also aid matching. We have a function for this in the linkage module. 

In [ ]:
df1 = linkage.alpha_name(df1, col_in = 'FORENAME', col_out = 'ALPHABETISE_FORENAME')
df2 = linkage.alpha_name(df2, col_in = 'FORENAME', col_out = 'ALPHABETISE_FORENAME')

df1.select("FORENAME", "ALPHABETISE_FORENAME").show()

There are more common matching variables we could still derive. Taking the **substring()** of our postcode column can help us derive less granular geographic variables for matching:

In [ ]:
df1 = dataframes.substring(df1, out_col = "PC_DISTRICT", target_col = "POSTCODE", start = 0, length = 4)
df2 = dataframes.substring(df2, out_col = "PC_DISTRICT", target_col = "POSTCODE", start = 0, length = 4)

df1.select("POSTCODE", "PC_DISTRICT").show()

If you have a time lag between the collection of two surveys you are trying to link together, you may want to align respondent ages for matching. We can do this using the age_at() function:

In [ ]:
# we can find out their age at the most recent Census, for example:
census_date = '2021-03-21'

df1 = standardisation.at_age(df1, birth_date = 'DOB', *age_at_dates = census_date, in_date_format = 'yyyy-MM-dd')
df2 = standardisation.at_age(df2, birth_date = 'DOB', *age_at_dates = census_date, in_date_format = 'yyyy-MM-dd')

df1.select('DOB','age_at_2021-03-21')

In [1]:
# NOT SURE IF WE NEED THE BELOW - JUST TAKING IT FROM DAP VERSION BEFORE DELETED

# Linking

First, let's create extra matching variables:

In [ ]:
# create full name 
df1 = dataframes.concat(df = df1, out_col = 'Full_Name', sep = ' ', cols = ['First_Name','Middle_Name','Last_Name'])
df2 = dataframes.concat(df = df2, out_col = 'Full_Name', sep = ' ', cols = ['First_Name','Middle_Name','Last_Name'])

df1.select('Full_Name').show()

In [ ]:
# alphaname can be useful to match on if characters have become swapped around, i.e. due to typing error 
df1 = linkage.alpha_name(df = df1, input_col = 'First_Name', output_col = 'alphaname')
df2 = linkage.alpha_name(df = df2, input_col = 'First_Name', output_col = 'alphaname')

df1.select('alpha_name').show()

In [ ]:
# forename metaphone - this is a phonetic encoder, and aims to capture codes that distinguish betweeen how words sound 
# so should deal well with names that have been interpreted incorrectly by field interviewers
df1 = df1.withColumn('forename_metaphone', linkage.metaphone(F.col('First_Name')))
df2 = df2.withColumn('forename_metaphone', linkage.metaphone(F.col('First_Name')))

df1.select('forename_metaphone').show()

# Deduplication

This is quite easily done, defining our duplicate matchkey(s) and using the **deduplicate** function:

In [ ]:
# define our matchkey
deduplicate_mkey = ['First_Name', 'Last_Name','Resident_Age','Sex','Postcode','Address']
df1.count()

In [ ]:
df1 = linkage.deduplicate(df = df1, record_id - 'Resident_ID', mks = deduplicate_mkey)
df2 = linkage.deduplicate(df = df2, record_id - 'Resident_ID', mks = deduplicate_mkey)
df1.count()

Now that we've removed duplicates, we can start to investigate some matchkeys:

In [ ]:
# first, let's suffix each dataset's columns to distinguish the two dataframes 
df1 = dataframes.suffix_columns(df1, suffix = '_df1')
df2 = dataframes.suffix_columns(df2, suffix = '_df2')

df1.persist().count()
df2.persist().count()

In [ ]:
MK1 = [df1.Full_Name_df1 == df2.Full_Name_df2,
       df1.Sex_df1 == df2.Sex_df2,
       df1.Resident_Age_df1 == df2.Resident_Age_df2,
       df1.Postcode_df1 == df2.Postcode_df2,]

# letting middle name be a mismatch 
MK2 = [df1.First_Name_df1 == df2.First_Name_df2,
       df1.Last_Name_df1 == df2.Last_Name_df2,
       df1.Sex_df1 == df2.Sex_df2,
       df1.Resident_Age_df1 == df2.Resident_Age_df2,
       df1.Postcode_df1 == df2.Postcode_df2,]

# taking the phonetic encoding of forename - using the metaphone algorithm
MK3 = [df1.forename_metaphone_df1 == df2.forename_metaphone_df2,
       df1.Last_Name_df1 == df2.Last_Name_df2,
       df1.Sex_df1 == df2.Sex_df2,
       df1.Resident_Age_df1 == df2.Resident_Age_df2,
       df1.Postcode_df1 == df2.Postcode_df2,]

# Now allowing for misspellings rather than mishearings of names, using standardised Levenshtein edit distance
MK4 = [linkage.std_lev_score(F.col('First_Name_df1'),F,col('First_Name_df2')) > 0.7,
       df1.Last_Name_df1 == df2.Last_Name_df2,
       df1.Sex_df1 == df2.Sex_df2,
       df1.Resident_Age_df1 == df2.Resident_Age_df2,
       df1.Postcode_df1 == df2.Postcode_df2,]

# similar to the above, but now using a different string comparison algorithm - the Jaro comparator
MK5 = [linkage.jaro(F.col('First_Name_df1'),F,col('First_Name_df2')) > 0.7,
       df1.Last_Name_df1 == df2.Last_Name_df2,
       df1.Sex_df1 == df2.Sex_df2,
       df1.Resident_Age_df1 == df2.Resident_Age_df2,
       df1.Postcode_df1 == df2.Postcode_df2,]

matchkeys = [MK1,MK2,MK3,MK4,MK5]

In [ ]:
links = linkage.deterministic_linkage(df_l = df1, df_r = df2, id_l = 'Resident_ID_df1', id_r = 'Resident_ID_df2', 
                                      matchkeys = matchkeys, our_dir = '/user/edwara5/df1_df2_links')

In [ ]:
links.show()